In [1]:
# This is a copy of my private kaggle kernel: https://www.kaggle.com/joatom/structures
# fork of https://www.kaggle.com/inversion/atomic-distance-benchmark/

#J-Coupling:https://www.youtube.com/watch?v=vnkk3eli1Hc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm_notebook as tqdm

import gc
import psutil
import os
import time
from functools import reduce

print(os.listdir("../input"))

['potential_energy.csv', 'mulliken_charges.csv', 'train.csv', 'scalar_coupling_contributions.csv', 'sample_submission.csv', 'structures', 'test.csv', 'magnetic_shielding_tensors.csv', 'dipole_moments.csv', 'structures.csv']


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

    

# General Functions

In [3]:
in_path='../input/'

In [4]:
# https://www.kaggle.com/todnewman/keras-neural-net-for-champs/comments
def show_ram_usage():
    py = psutil.Process(os.getpid())
    print('RAM usage: {} GB'.format(py.memory_info()[0]/2. ** 30))

In [5]:
# https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655#latest-565815
# guiferviz comment

def reduce_memory_usage(df, deep=True, verbose=True, categories=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = df.memory_usage().sum() / 1024**2

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if col_type == "object":
            df[col] = df[col].astype("category")
            best_type = "category"
        elif col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    if verbose:
        end_mem = df.memory_usage().sum() / 1024**2
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        print(f"Memory usage decreased from"
              f" {start_mem:.2f}MB to {end_mem:.2f}MB"
              f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")
    return df

# Structures Data

In [6]:
structures = pd.read_csv(in_path+'structures.csv')
display(structures.head())

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [7]:
structures.groupby(by='molecule_name').count().describe()

,atom_index,atom,x,y,z
count,130775.000000,130775.000000,130775.000000,130775.000000,130775.000000
mean,18.035993,18.035993,18.035993,18.035993,18.035993
std,2.938363,2.938363,2.938363,2.938363,2.938363
min,3.000000,3.000000,3.000000,3.000000,3.000000
25%,16.000000,16.000000,16.000000,16.000000,16.000000
50%,18.000000,18.000000,18.000000,18.000000,18.000000
75%,20.000000,20.000000,20.000000,20.000000,20.000000
max,29.000000,29.000000,29.000000,29.000000,29.000000


In [8]:
gc.collect()
#structures.info()
show_ram_usage()

RAM usage: 0.2698249816894531 GB


In [9]:
%%time
def norm_std(df, col):
    df[col+'_norm_std'] = ((df[col]-df.groupby('molecule_name')[col].transform(np.mean)) / df.groupby('molecule_name')[col].transform(np.std)).fillna(df[col]-df.groupby('molecule_name')[col].transform(np.mean))
    return df


def norm_minmax(df, col):
    df[col+'_norm_minmax'] = ((df[col]-df.groupby('molecule_name')[col].transform(np.min)) / (df.groupby('molecule_name')[col].transform(np.max)-df.groupby('molecule_name')[col].transform(np.min))).fillna(df[col]-df.groupby('molecule_name')[col].transform(np.min))
    return df

#structures = norm_std(structures, 'x') 
#structures = norm_std(structures, 'y') 
#structures = norm_std(structures, 'z') 

#structures = norm_minmax(structures, 'x') 
#structures = norm_minmax(structures, 'y') 
#structures = norm_minmax(structures, 'z') 

'number of molecules:', structures['molecule_name'].unique().shape

CPU times: user 116 ms, sys: 28 ms, total: 144 ms
Wall time: 147 ms


('number of molecules:', (130775,))

# Normalization of spartial data

In [10]:
structures=reduce_memory_usage(structures)
gc.collect()
show_ram_usage()

structures.head(n=10)
#structures

Column 'molecule_name' converted from object to category
Column 'atom_index' converted from int64 to int8
Column 'atom' converted from object to category
Column 'x' converted from float64 to float32
Column 'y' converted from float64 to float32
Column 'z' converted from float64 to float32
Memory usage decreased from 107.97MB to 46.49MB (61.48MB, 56.94% reduction)
RAM usage: 0.22905349731445312 GB


,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [15]:
k=10

In [16]:
%%time
import warnings  
warnings.filterwarnings('ignore')

def nn_dists(l):
    
    nn = k+1
    x=np.array(structures.loc[l.index,'x'])
    y=np.array(structures.loc[l.index,'y'])
    z=np.array(structures.loc[l.index,'z'])
    coord = np.append(np.append(x,y),z).reshape((l.size,3),order='F')
    
    nbrs = NearestNeighbors(n_neighbors=min(len(coord),nn), algorithm='ball_tree').fit(coord)
    distances, indices = nbrs.kneighbors(coord) 
    
    # print(len(indices),indices.shape)    
    fb_dist=0
    
    if indices.shape != (1,1):
        # PCA
        pca = PCA(n_components=2)
        p=pca.fit_transform(coord)
        
        # NN and NN distance
        atm_idx = np.pad(indices[:,1:l.size],((0,0),(0, max(0, nn-l.size))), 'constant', constant_values=(9999, 9999))
        dst = np.pad(distances[:,1:l.size], ((0,0),(0,max(0,nn-l.size))), 'constant', constant_values=(fb_dist, fb_dist))
        #print('p',p,'x',x.reshape(l.size,1),'d',dst)
        
        # LookUps
        lu = np.append(np.array(structures.loc[l.index,'atom']),np.array('N/A'))
        lu_x = np.append(np.array(structures.loc[l.index,'x']),np.array(fb_dist))
        lu_y = np.append(np.array(structures.loc[l.index,'y']),np.array(fb_dist))
        lu_z = np.append(np.array(structures.loc[l.index,'z']),np.array(fb_dist))
        
        nn_x = np.take(lu_x, atm_idx, mode = 'clip') 
        nn_y = np.take(lu_y, atm_idx, mode = 'clip') 
        nn_z = np.take(lu_z, atm_idx, mode = 'clip') 
        atm = np.take(lu, atm_idx, mode = 'clip')
    else: 
        #p = np.ones((1, 2))*(999)
        atm_idx = np.ones((1, max(0, nn-l.size)))*(9999) 
        atm = np.ones((1, max(0, nn-l.size)))*(9999) 
        dst = np.ones((1, max(0, nn-l.size)))*(9999)
        nn_x = np.ones((1, max(0, nn-l.size)))*(9999)
        nn_y = np.ones((1, max(0, nn-l.size)))*(9999)
        nn_z = np.ones((1, max(0, nn-l.size)))*(9999)
        
        
    #out = np.append(np.append(np.append(np.append(np.append(atm,dst,axis=1),nn_x, axis=1),nn_y, axis=1),nn_z, axis=1) ,atm_idx, axis=1)
    out = np.append(np.append(np.append(np.append(np.append(np.append(atm,dst,axis=1),nn_x, axis=1),nn_y, axis=1),nn_z, axis=1) ,atm_idx, axis=1),p,axis=1)
    #out = np.append(atm,dst,axis=1)    
        
    #print('x',x,'lux',structures.loc[l.index,'x'],'nn_x',nn_x,'out:', out)
    
    #print('###')
    return [i for i in out]#np.squeeze(distances[:,1:2])

structures['num_atoms_in_mol'] = structures.groupby('molecule_name')['x'].transform('count')
structures['nearestn'] = structures.groupby('molecule_name')['x'].transform(nn_dists)



#11mi 12s

CPU times: user 12min 51s, sys: 4.74 s, total: 12min 56s
Wall time: 12min 56s


In [17]:
%%time
gc.collect()

for i in range(k):
    structures['nn_'+str(i+1)] = structures['nearestn'].apply(lambda x: x[k*0+i])
    structures['nn_'+str(i+1)+'_dist'] = structures['nearestn'].apply(lambda x: x[k*1+i])
    structures['nn_x_'+str(i+1)] = structures['nearestn'].apply(lambda x: x[k*2+i])  - structures['x']
    structures['nn_y_'+str(i+1)] = structures['nearestn'].apply(lambda x: x[k*3+i])  - structures['y']
    structures['nn_z_'+str(i+1)] = structures['nearestn'].apply(lambda x: x[k*4+i])  - structures['z']
    structures['nn_idx_'+str(i+1)] = structures['nearestn'].apply(lambda x: x[k*5+i])


structures['pca_x'] = structures['nearestn'].apply(lambda x: x[-2])
structures['pca_y'] = structures['nearestn'].apply(lambda x: x[-1])

# 67sec

CPU times: user 1min 47s, sys: 2.75 s, total: 1min 49s
Wall time: 1min 49s


In [18]:
#https://www.kaggle.com/todnewman/keras-neural-net-for-champs/comments
# center of molecule
#structures['c_mol_x']=structures.groupby('molecule_name')['x_norm_minmax'].transform('mean')
#structures['c_mol_y']=structures.groupby('molecule_name')['y_norm_minmax'].transform('mean')
#structures['c_mol_z']=structures.groupby('molecule_name')['z_norm_minmax'].transform('mean')

structures['c_mol_x']=structures.groupby('molecule_name')['x'].transform('mean')
structures['c_mol_y']=structures.groupby('molecule_name')['y'].transform('mean')
structures['c_mol_z']=structures.groupby('molecule_name')['z'].transform('mean')



In [19]:
structures.head(n=5)

,molecule_name,atom_index,atom,x,y,z,num_atoms_in_mol,nearestn,nn_1,nn_1_dist,...,nn_10_dist,nn_x_10,nn_y_10,nn_z_10,nn_idx_10,pca_x,pca_y,c_mol_x,c_mol_y,c_mol_z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,5,"[H, H, H, H, N/A, N/A, N/A, N/A, N/A, N/A, 1.0...",H,1.091946,...,0.0,0.012698,-1.085804,-0.008001,9999,-0.000012,-1.998521e-07,-0.012689,1.085797,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,5,"[C, H, H, H, N/A, N/A, N/A, N/A, N/A, N/A, 1.0...",C,1.091953,...,0.0,-0.002150,0.006031,-0.001976,9999,0.620612,-2.676423e-01,-0.012689,1.085797,0.008001
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,5,"[C, H, H, H, N/A, N/A, N/A, N/A, N/A, N/A, 1.0...",C,1.091952,...,0.0,-1.011731,-1.463751,-0.000277,9999,0.640181,2.553965e-01,-0.012689,1.085797,0.008001
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,5,"[C, H, H, H, N/A, N/A, N/A, N/A, N/A, N/A, 1.0...",C,1.091946,...,0.0,0.540815,-1.447527,0.876644,9999,-0.636446,-8.461912e-01,-0.012689,1.085797,0.008001
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,5,"[C, H, H, H, N/A, N/A, N/A, N/A, N/A, N/A, 1.0...",C,1.091948,...,0.0,0.523814,-1.437933,-0.906397,9999,-0.624336,8.584373e-01,-0.012689,1.085797,0.008001


In [20]:
#structures = structures.drop(columns='nearestn',axis=0)
structures.drop(columns='nearestn',axis=0, inplace=True)
gc.collect()


126

In [21]:
#structures = pd.read_csv('../input/v13molecular/struct_ext.csv') 
#display(structures.head())

In [22]:
%%time
atom_map={'H': 1, 'C': 6, 'N':7,'O':8,'F':9, 'N/A':9999}
structures['atom'] = structures['atom'].map(atom_map)
for i in range(1,k+1):
    structures['nn_'+str(i)] = structures['nn_'+str(i)].map(atom_map)

structures.head()

CPU times: user 2.43 s, sys: 748 ms, total: 3.18 s
Wall time: 3.18 s


,molecule_name,atom_index,atom,x,y,z,num_atoms_in_mol,nn_1,nn_1_dist,nn_x_1,...,nn_10_dist,nn_x_10,nn_y_10,nn_z_10,nn_idx_10,pca_x,pca_y,c_mol_x,c_mol_y,c_mol_z
0,dsgdb9nsd_000001,0,6,-0.012698,1.085804,0.008001,5,1,1.091946,-0.528117,...,0.0,0.012698,-1.085804,-0.008001,9999,-0.000012,-1.998521e-07,-0.012689,1.085797,0.008001
1,dsgdb9nsd_000001,1,1,0.002150,-0.006031,0.001976,5,6,1.091953,-0.014849,...,0.0,-0.002150,0.006031,-0.001976,9999,0.620612,-2.676423e-01,-0.012689,1.085797,0.008001
2,dsgdb9nsd_000001,2,1,1.011731,1.463751,0.000277,5,6,1.091952,-1.024429,...,0.0,-1.011731,-1.463751,-0.000277,9999,0.640181,2.553965e-01,-0.012689,1.085797,0.008001
3,dsgdb9nsd_000001,3,1,-0.540815,1.447527,-0.876644,5,6,1.091946,0.528117,...,0.0,0.540815,-1.447527,0.876644,9999,-0.636446,-8.461912e-01,-0.012689,1.085797,0.008001
4,dsgdb9nsd_000001,4,1,-0.523814,1.437933,0.906397,5,6,1.091948,0.511115,...,0.0,0.523814,-1.437933,-0.906397,9999,-0.624336,8.584373e-01,-0.012689,1.085797,0.008001


In [23]:
#%%time

#cat_names = ['nn_'+str(i) for i in range(1,k+1)]

#for c in cat_names:
#    lbl = LabelEncoder()
#    structures[c] = lbl.fit_transform(list(structures[c].values))

In [24]:
structures['nn_7'].unique()

array([9999,    1,    6,    8,    7,    9])

In [25]:
#structures.to_csv('temp_struct.csv', index=False)

structures=reduce_memory_usage(structures)
gc.collect()
show_ram_usage()
structures.head(n=5)

Column 'atom' converted from int64 to int8
Column 'num_atoms_in_mol' converted from int64 to int8
Column 'nn_1' converted from int64 to int8
Column 'nn_1_dist' converted from float64 to float32
Column 'nn_x_1' converted from float64 to float32
Column 'nn_y_1' converted from float64 to float32
Column 'nn_z_1' converted from float64 to float32
Column 'nn_idx_1' converted from int64 to int8
Column 'nn_2' converted from int64 to int8
Column 'nn_2_dist' converted from float64 to float32
Column 'nn_x_2' converted from float64 to float32
Column 'nn_y_2' converted from float64 to float32
Column 'nn_z_2' converted from float64 to float32
Column 'nn_idx_2' converted from int64 to int8
Column 'nn_3' converted from int64 to int16
Column 'nn_3_dist' converted from float64 to float32
Column 'nn_x_3' converted from float64 to float32
Column 'nn_y_3' converted from float64 to float32
Column 'nn_z_3' converted from float64 to float32
Column 'nn_idx_3' converted from int64 to int16
Column 'nn_4' convert

,molecule_name,atom_index,atom,x,y,z,num_atoms_in_mol,nn_1,nn_1_dist,nn_x_1,...,nn_10_dist,nn_x_10,nn_y_10,nn_z_10,nn_idx_10,pca_x,pca_y,c_mol_x,c_mol_y,c_mol_z
0,dsgdb9nsd_000001,0,6,-0.012698,1.085804,0.008001,5,1,1.091946,-0.528117,...,0.0,0.012698,-1.085804,-0.008001,9999,-0.000012,-1.998521e-07,-0.012689,1.085797,0.008001
1,dsgdb9nsd_000001,1,1,0.002150,-0.006031,0.001976,5,6,1.091953,-0.014849,...,0.0,-0.002150,0.006031,-0.001976,9999,0.620612,-2.676423e-01,-0.012689,1.085797,0.008001
2,dsgdb9nsd_000001,2,1,1.011731,1.463751,0.000277,5,6,1.091952,-1.024429,...,0.0,-1.011731,-1.463751,-0.000277,9999,0.640181,2.553965e-01,-0.012689,1.085797,0.008001
3,dsgdb9nsd_000001,3,1,-0.540815,1.447527,-0.876644,5,6,1.091946,0.528117,...,0.0,0.540815,-1.447527,0.876644,9999,-0.636446,-8.461912e-01,-0.012689,1.085797,0.008001
4,dsgdb9nsd_000001,4,1,-0.523814,1.437933,0.906397,5,6,1.091948,0.511115,...,0.0,0.523814,-1.437933,-0.906397,9999,-0.624336,8.584373e-01,-0.012689,1.085797,0.008001


# predict mulliken_charges

In [26]:
#structures.to_csv('struct_ext.csv', index = False)
structures.to_pickle('struct_ext.zip', compression='zip')

## Count number of atoms per molecule

In [27]:
'''atoms_in_molecule = structures.groupby(['molecule_name','atom']).count()[['x']].unstack(fill_value=0).fillna(0)
atoms_in_molecule.columns=atoms_in_molecule.columns.droplevel()

atoms_in_molecule=reduce_memory_usage(atoms_in_molecule)
gc.collect()
show_ram_usage()

atoms_in_molecule.head()'''

"atoms_in_molecule = structures.groupby(['molecule_name','atom']).count()[['x']].unstack(fill_value=0).fillna(0)\natoms_in_molecule.columns=atoms_in_molecule.columns.droplevel()\n\natoms_in_molecule=reduce_memory_usage(atoms_in_molecule)\ngc.collect()\nshow_ram_usage()\n\natoms_in_molecule.head()"

In [28]:
#atoms_in_molecule.to_csv('atoms_in_mol.csv', index = False)